In [1]:
#visualisation and work with dfs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, TimeDistributed, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

%matplotlib inline

Using TensorFlow backend.


# Data preparations

In [2]:
#Load the data into Pandas dataframe
data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
data3 = pd.read_csv('../task3_data/task3_training.tsv', sep='\t')[['type', 'tweet']]

# Vectorization by keras

# Vectorization by fasttext

In [3]:
import fasttext

max_words = 10000
max_len = 50


data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df = data2[:5000]

df_ft = df.iloc[np.random.permutation(len(df))]
df_ft['class'] = df_ft['class'].apply(lambda x: '__label__' + str(x))

df_train, df_test = train_test_split(df_ft , test_size=0.2)

df_train[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")
df_test[['class', 'tweet']].to_csv('TEST.txt', header=False, index=False, sep="\t")
#print(df_train, df_test)

graph = []
maxima, maxima_idx = 0, 0

# просто отрабатывает модель с классификацией, но я просто забираю потом отсюда вектора слов 
for epoch in range(15):
    model = fasttext.train_supervised(input='TRAIN.txt', epoch=epoch)
    All, Pos = 0, 0
    for (clas, tweet) in zip(df_test['class'], df_test['tweet']):
        All += 1
        if int(clas[-1]) == int(model.predict(tweet, k=2)[0][0][-1]): 
            #print(int(clas[-1]),int(model.predict(tweet, k=2)[0][0][-1]))
            Pos += 1
    graph.append(Pos/All)
    if Pos/All > maxima: maxima, maxima_idx = Pos/All, epoch

In [4]:
#Split into training and test data.
X_train = df_train['tweet']
X_test = df_test['tweet']
y_train = df_train['class']
y_test = df_test['class']

y_train = y_train.apply(lambda x: x[-1])
y_test = y_test.apply(lambda x: x[-1])

# it's special. we can't work with other format
X_train = np.array(X_train)
X_test = np.array(X_test)

#по сути на этом этапе можно слегка почистить составляющие твитов
for i in range(len(X_train)):
    sentence = X_train[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_train[i] = sentence

for i in range(len(X_test)):
    sentence = X_test[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_test[i] = sentence

maxima = 0
for ar in X_train:
    if len(ar)>maxima: maxima = len(ar)
for ar in X_test:
    if len(ar)>maxima: maxima = len(ar)
max_len = maxima + 1
X_train.shape

(4000,)

In [ ]:
pd.DataFrame(X_train.tolist())

In [5]:
#ars = [[1,2,3], [4,5,6,7], [9,1]]

def add_pads(ars, max_len):
#     lenn = 0
#     for ar in ars: 
#         if len(ar)>lenn: lenn = len(ar)

    for i in range(len(ars)):
        while len(ars[i]) < max_len: ars[i].insert(0,np.array([0 for i in range(100)]))
    return ars

X_train = add_pads(X_train, max_len)
X_test = add_pads(X_test, max_len)

In [6]:
X_train = X_train.tolist()
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[-0.004066091, 0.009607123, 0.007753223, 0.004...","[0.002317975, 0.004780741, 0.0022626508, -0.00...","[-0.001364779, -0.0026793578, -0.0015270945, -...","[0.0066822204, -0.0060909805, -0.007965945, 0....","[0.0030976562, 0.036932245, -0.01184896, -0.01...","[-0.008325465, 0.009131809, 0.0010765668, 0.00...","[-0.008756492, -0.013557038, 0.009647012, 0.01...","[0.016990144, 0.049457714, -0.013272082, -0.03...","[0.009791253, 0.019368328, -0.012212653, -0.00...","[-0.004066091, 0.009607123, 0.007753223, 0.004..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[-0.019549407, -0.090289064, 0.03336306, 0.055...","[0.0016194505, -6.0688944e-05, 0.01016432, -0....","[0.003604437, 0.0035918013, -0.005115295, -0.0...","[0.0057966155, -0.010592762, -0.009029159, -0....","[0.0070699276, 0.018394142, -0.000521627, -0.0...","[-0.0078392, -0.007908128, -0.0061241263, -0.0...","[-0.0019632324, 0.026374592, -0.014950188, -0....","[-0.0010868062, 0.005450878, 0.0031571207, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0016207944, 0.023197126, 0.0026727496, -0.0..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.0130255995, 0.04509486, -0.015660625, -0.02...","[-0.0037613073, 0.02611134, -0.0086502405, -0....","[0.013805779, 0.011874609, -0.007331133, -0.02...","[0.0069102463, 0.07563372, -0.028806403, -0.05...","[0.011166625, -0.0013266408, -0.004458989, -0....","[0.0030976562, 0.036932245, -0.01184896, -0.01...","[-0.010144113, -0.05656143, 0.025894944, 0.021...","[-0.0077478206, 0.01495181, -0.007625238, 0.00...","[-0.00077580067, 0.031240383, -0.013076901, -0...","[-0.0058562546, -0.045964975, 0.016610002, 0.0..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.0026265054, -0.0010077859, 0.010089649, 0.0...","[0.010515793, 0.012750229, -0.0007932945, -0.0...","[0.0069536543, 0.0497909, -0.019917186, -0.029...","[-0.0016239426, 0.0104538705, -0.009502799, -0...","[0.016990144, 0.049457714, -0.013272082, -0.03...","[0.004386101, 0.0167

In [7]:
X = np.array(X_train).reshape(len(X_train), len(X_train[1]), 100)

In [8]:
X.shape

(4000, 63, 100)

In [9]:
X_train = np.array(X_train)

# Configure LSTM

In [17]:
# configure network
#n_batch = len(X) # !!!!!
n_batch = 50
n_epoch = 10
n_neurons = 300

# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])


model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (50, 300)                 481200    
_________________________________________________________________
dense_4 (Dense)              (50, 1)                   301       
Total params: 481,501
Trainable params: 481,501
Non-trainable params: 0
_________________________________________________________________


# Fit models

In [18]:
# fit network
for i in range(n_epoch):
    model.fit(X_train, y_train, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.1624 - acc: 0.9538
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0495 - acc: 0.9843
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0502 - acc: 0.9870
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0476 - acc: 0.9883
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0286 - acc: 0.9912
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0388 - acc: 0.9890
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0298 - acc: 0.9908
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0297 - acc: 0.9900
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0247 - acc: 0.9925
Epoch 1/1
4000/4000 [==============================] - 10s 2ms/step - loss: 0.0442 - acc: 0.9895


In [19]:
# аналогично X_train подготовлен X_test
X_test = X_test.tolist()
X_test = np.array(X_test).reshape(len(X_test), len(X_test[1]), 100)
X_test = np.array(X_test)
X_test.shape
type(X_test)

numpy.ndarray

In [20]:
# перекидываем веста в новую модель для размера батча 1
n_batch = 1

# re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)

# compile model
new_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

# Evaluate

In [21]:
for i in range(1):
    testX, testy = np.array(X_train.tolist()[i]), np.array(y_test.tolist()[i])
    testX = testX.reshape(1, 63, 100)
    yhat = new_model.predict(testX, batch_size=1)
    print(f'>Expected={y_test.tolist()[i]}, Predicted={yhat}')

>Expected=0, Predicted=[[-1.58076920e-02 -2.43283555e-01 -1.24918679e-02 -2.09215395e-02
  -1.25595689e-01  4.61456776e-02 -2.50635743e-01 -4.79658097e-01
   3.74115109e-02  1.02818564e-01 -4.38308388e-01 -1.74328759e-01
   3.21503460e-01 -2.67082863e-02  1.19942710e-01 -9.47263464e-02
  -6.16565168e-01 -5.38080454e-01  3.99870984e-03  4.65076417e-01
   1.86209589e-01 -6.11212775e-02  6.34380519e-01  1.68295223e-02
  -1.35362372e-01  3.90656181e-02  4.40383367e-02  3.82993557e-02
  -4.68352199e-01 -2.14694832e-02 -5.99275790e-02  6.68917716e-01
   1.74229547e-01  2.30767410e-02  3.96613628e-01  3.57231975e-01
  -1.77239273e-02  6.17806353e-02 -2.94135809e-02 -3.22941840e-02
  -6.88730031e-02 -7.58526102e-02  8.85059033e-03 -2.20867223e-03
   3.61479213e-03 -3.32266395e-03  2.60412134e-02 -2.85364334e-02
  -1.07323542e-01  1.46223769e-01  5.57433814e-03  3.58843245e-02
   5.19659072e-02 -4.15413752e-02 -1.29882783e-01  8.12407583e-03
   1.73332855e-01 -2.07924455e-01  1.13047017e-02 -4.

In [ ]:
scores = new_model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# for i, j in zip(model.predict(test_sequences_matrix), Y_test):
#     print(i, j)
list(model1.predict(X_test))
predicts = [i[0] for i in model1.predict(X_test)]

gap = (max(predicts)-min(predicts))/2
for i in range(len(predicts)):
    if i <= gap: predicts[i] = 0
    else: predicts[i] = 1

from sklearn.metrics import classification_report
print(classification_report(y_test, predicts))
      

#f1_score(Y_test, predicts, average='micro')

In [ ]:
# Evaluate the model on the test set
accr = model.evaluate(test_sequences_matrix, Y_test)